# Throughput measurements for warehouse_4 workers

Conclusions:
- These are already analyzed in some word document?
- Scales at least up to 30 workers
- 10k resolution shows less improvement between 10 and 30 workers, indicating near saturation

Missing plots:
- Node-to-node variance (is there any?)

Future experiments:
-

In [7]:
import common_utils
import os
import pandas as pd

# Example usage
root_folder = '../../../data_warehouse/minimized_warehouse_5'
filename = 'worker1.feather'
subfolders = common_utils.find_subfolders_with_file(root_folder, filename)
print(subfolders)
prom_data_paths = {os.path.basename(x): x for x in subfolders}
yolo_data_paths = {key: os.path.join(val, "worker_qos.feather") for key, val in prom_data_paths.items()}


['../../../data_warehouse/minimized_warehouse_5/1737742814_(1.5000)', '../../../data_warehouse/minimized_warehouse_5/1737740010_(1.1000)', '../../../data_warehouse/minimized_warehouse_5/1737672502_(9.1000)', '../../../data_warehouse/minimized_warehouse_5/1737709145_(5.5000)', '../../../data_warehouse/minimized_warehouse_5/1737725976_(3.5000)', '../../../data_warehouse/minimized_warehouse_5/1737695091_(7.10000)', '../../../data_warehouse/minimized_warehouse_5/1737714755_(4.1000)', '../../../data_warehouse/minimized_warehouse_5/1737703532_(6.10000)', '../../../data_warehouse/minimized_warehouse_5/1737666819_(10.5000)', '../../../data_warehouse/minimized_warehouse_5/1737692273_(7.5000)', '../../../data_warehouse/minimized_warehouse_5/1737717559_(4.5000)', '../../../data_warehouse/minimized_warehouse_5/1737678132_(9.10000)', '../../../data_warehouse/minimized_warehouse_5/1737675307_(9.5000)', '../../../data_warehouse/minimized_warehouse_5/1737689473_(7.1000)', '../../../data_warehouse/mini

In [8]:
"""
Get corresponding yolo stats for each model 
"""
response_time = {}
for key in prom_data_paths.keys():
    try:
        yolo_df = common_utils.read_feather_cached(yolo_data_paths[key])
    except:
        print(f"Failed to read {key}")
        continue
    yolo_df['total_inference_time'] = yolo_df['inf'] + yolo_df['post'] + yolo_df['pre']
    yolo_df['end_to_end_response_time'] = yolo_df['total_inference_time'] + yolo_df['queue']
    yolo_df['end'] = pd.to_datetime(yolo_df['end_time'], unit='ms')  # Convert to datetime (optional)
    yolo_df.set_index('end', inplace=True)
    resampled_df = yolo_df.resample('5s')
    model_info = common_utils.path_to_workers_and_pcl_size(key)
    if model_info.resolution not in response_time:
        response_time[model_info.resolution] = {}
    response_time[model_info.resolution][model_info.num_vehicles] = resampled_df.agg({'end_to_end_response_time': 'count'}).reset_index()['end_to_end_response_time'].rename(key) / 5

In [9]:
import plotly.express as px

for resolution in sorted(response_time.keys()):
    queue_df = pd.DataFrame.from_dict(response_time[resolution])
    fig = px.line(queue_df, x=queue_df.index, y=queue_df.columns)
    fig.update_layout(title=f'PCL throughput (Resolution: {resolution})', xaxis_title='Time', yaxis_title='PCL per second', yaxis_type='linear')
    fig.show()

In [10]:
for resolution in sorted(response_time.keys()):
    queue_df = pd.DataFrame.from_dict(response_time[resolution])
    print(f"\nResolution: {resolution}")
    print("num_workers, max_throughput")
    print(queue_df.max())


Resolution: 1000
num_workers, max_throughput
1        4.8
9       36.8
4       17.6
7       29.0
2        9.6
5       21.0
3       12.8
8       33.6
10      41.6
20      78.6
6       25.6
30    1134.2
dtype: float64

Resolution: 5000
num_workers, max_throughput
1     1.6
5     1.6
3     1.6
10    3.2
7     1.6
4     1.6
9     1.6
20    4.8
30    4.8
6     1.6
8     1.6
2     1.6
dtype: float64

Resolution: 10000
num_workers, max_throughput
7     1.6
6     1.6
9     1.6
5     1.6
4     1.6
1     1.6
30    1.6
10    1.6
3     1.6
20    1.6
8     1.6
2     1.6
dtype: float64


In [11]:

import plotly.express as px
import pandas as pd

# Dictionary to store max throughput for each model
max_throughput = {}

for resolution in sorted(response_time.keys()):
    queue_df = pd.DataFrame.from_dict(response_time[resolution])
    
    # Calculate the maximum throughput for each model
    max_throughput[resolution] = queue_df.max()

# Create a DataFrame from the max throughput dictionary
max_throughput_df = pd.DataFrame(max_throughput)

# Sort 
max_throughput_df_sorted = common_utils.sort_by_model_size_then_version(max_throughput_df)

# Plot the maximum throughput for each model
fig = px.bar(max_throughput_df_sorted, barmode='group')
fig.update_layout(title='Maximum Throughput per Model', xaxis_title='Model', yaxis_title='Max Images per Second', yaxis_type='linear')
fig.show()

fig = px.bar(max_throughput_df_sorted, barmode='group')
fig.update_layout(title='Maximum Throughput per Model', xaxis_title='Model', yaxis_title='Max Images per Second', yaxis_type='log')
fig.show()

AttributeError: 'int' object has no attribute 'replace'

In [6]:
max_throughput_df_sorted.to_csv("table_plots/data_throughput.csv")

OSError: Cannot save file into a non-existent directory: 'table_plots'

In [ ]:
max_throughput_df_sorted